In [1]:
import sys
from pathlib import Path
import typer
import pandas as pd
from loguru import logger
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from loguru import logger

project_root = Path().resolve().parent
sys.path.append(str(project_root / "src"))

from entities.params import read_pipeline_params

In [2]:
params = read_pipeline_params('../params.yaml')

In [3]:
df = pd.read_csv('../' / Path(params.paths.raw_data_dir) / params.data.train_file)
df.dropna(inplace=True)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexx\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexx\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexx\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [5]:
def preprocess_text(text: str, remove_digits=True) -> str:
    if remove_digits:
        text = re.sub(r"[^a-zA-Z\s]", "", text)
    else:
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    text = text.lower()
    text = re.sub(r"\s+", " ", text).strip()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    text = " ".join(words)
    return text

In [6]:
df['comment_text'] = df['comment_text'].apply(preprocess_text)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww match background colour im seemingly stuc...,0,0,0,0,0,0
2,000113f07ec002fd,hey man im really trying edit war guy constant...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggestion improvement wondered...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time asking view completely contradicts...,0,0,0,0,0,0
159567,ffea4adeee384e90,ashamed horrible thing put talk page,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm there actual article prostitution ...,0,0,0,0,0,0
159569,fff125370e4aaaf3,look like actually put speedy first version de...,0,0,0,0,0,0


In [8]:
texts = df["comment_text"].dropna().tolist()
texts

['explanation edits made username hardcore metallica fan reverted werent vandalism closure gas voted new york doll fac please dont remove template talk page since im retired',
 'daww match background colour im seemingly stuck thanks talk january utc',
 'hey man im really trying edit war guy constantly removing relevant information talking edits instead talk page seems care formatting actual info',
 'cant make real suggestion improvement wondered section statistic later subsection type accident think reference may need tidying exact format ie date format etc later noone else first preference formatting style reference want please let know appears backlog article review guess may delay reviewer turn listed relevant form eg wikipediagoodarticlenominationstransport',
 'sir hero chance remember page thats',
 'congratulation well use tool well talk',
 'cocksucker piss around work',
 'vandalism matt shirvington article reverted please dont banned',
 'sorry word nonsense offensive anyway im in

In [9]:
from transformers import AutoModel, AutoTokenizer, AdamW
tokenizer = AutoTokenizer.from_pretrained(params.model.pretrained_model_name)

In [10]:
from statistics import mean, median
import numpy as np
encoded = [tokenizer.encode(t, add_special_tokens=True) for t in texts]

lengths = [len(e) for e in encoded]

avg_length = mean(lengths)
median_length = median(lengths)
max_length_value = max(lengths)
quantile_90 = np.quantile(lengths, 0.9)

print(f"Average length: {avg_length}")
print(f"Median length: {median_length}")
print(f"Max length: {max_length_value}")
print(f"90th percentile: {quantile_90}")

Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


Average length: 51.49114814095293
Median length: 28
Max length: 2724
90th percentile: 111.0


In [ ]:
train_data, val_data = train_test_split(
    df,
    test_size=1 - params.data.train_split,
    random_state=params.data.random_state
)

In [54]:
processed_data_dir = Path("../") / Path(params.paths.processed_data_dir)
processed_data_dir.mkdir(parents=True, exist_ok=True)

In [55]:
train_data.to_csv(processed_data_dir / "train.csv", index=False)
val_data.to_csv(processed_data_dir / "val.csv", index=False)

In [62]:
logger.info(f"Processed train and validation datasets saved in {processed_data_dir}")

2024-12-02 14:38:36.869 | INFO     | __main__:<module>:1 - Processed train and validation datasets saved in ..\data\processed
